In [1]:
#Coletando os dados e guardando no excel
import http.client 
import json
import pandas as pd


In [ ]:
cnpj_exemplo = "33157312000162"


def obter_dados_empresa_cnpj(cnpj):
    conexao = http.client.HTTPSConnection("www.receitaws.com.br")
    conexao.request("GET", f"/v1/cnpj/{cnpj}")

    resposta = conexao.getresponse()
    print(f"Status da Resposta HTTP: {resposta.status}")

    if resposta.status != 200:
        return {"Status": "ERROR", "Menssage": f"Resposta HTTP com status {resposta.status}"}
    
    dados = resposta.read()
    conexao.close()

    try:
        empresa = json.loads(dados.decode("utf-8"))
        print(f"Empresa decodificada: {empresa}")
        return empresa

    except json.JSONDecodeError as e:

        print(f"Erro na decodificação do JSON{str(e)}")
        return {"Status": "ERROR", "Menssage": "Erro na decodificação do json "}




dados_empresa = obter_dados_empresa_cnpj(cnpj_exemplo)
salvar_dados_empresa_excel(dados_empresa)